In [37]:
#| default_exp train.load_data

We don't have the full dataset however the partial dataset should be fine to protype on. We need to see how exactly we need to set up the data.

We have for our ingredients, the weight (which we're ignoring just now), and its molecule_df id. We need to quickly adjust the molecule_db matching to match directly with the tokens of it. 

# Setup

In [2]:
#| export
from pyprojroot import here
root = here()
import sys
sys.path.append(str(root))

In [3]:
#| export
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from operator import itemgetter
import random
from itertools import islice

In [4]:
ingredients_df = pd.read_feather(f'{root}/data/local/recipe/partial/ingredients/0.feather')
food_ids = pd.read_feather(f'{root}/data/local/molecule/partial/food_ids/1_na_filled.feather')

In [5]:
#| export 
food_compounds_df = pd.read_feather(f'{root}/data/local/molecule/full/food_compounds/0.feather')

In [6]:
#| export
special_token_idxs = {
    'pad': food_compounds_df.shape[0], 'unknown': food_compounds_df.shape[0]+1, 'mask': food_compounds_df.shape[0]+2
}

In [7]:
special_token_idxs

{'pad': 1106, 'unknown': 1107, 'mask': 1108}

In [8]:
food_ids = food_ids.astype('Int64')

In [9]:
ingredients_df.join(food_ids)[['food_id']].join(food_compounds_df, on=['food_id'])

food_id  108  136       236  237  247  248  249  307  312  \
recipe  ingredient                                                              
1746116 0               667  0.0  0.0  0.109333  0.0  0.0  0.0  0.0  0.0  0.0   
        1               670  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        2               633  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        3               195  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        4               825  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
...                     ...  ...  ...       ...  ...  ...  ...  ...  ...  ...   
931097  9               994  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        10               40  0.0  0.0       0.0  0.0  0.0  0.0  0.0  0.0  0.0   
        11              125  0.0  0.0  0.109333  0.0  0.0  0.0  0.0  0.0  0.0   
        12              334  0.0  0.0  0.109333  0.0  0.0  0.0  0.0  0.0  0.0   
        13              119  0.0  0.0  0.109333  0.0  0.0  0.0  0.0  0.0  0.0   

                    ...    122183    122207    122257    122370    122486  \
recipe  ingredient  ...                                                     
1746116 0           ...  23.49215  9.498727  2.253628  9.376545  9.376545   
        1           ...       0.0       0.0       0.0       0.0       0.0   
        2           ...       0.0       0.0       0.0       0.0       0.0   
        3           ...       0.0       0.0       0.0       0.0       0.0   
        4           ...       0.0       0.0       0.0       0.0       0.0   
...                 ...       ...       ...       ...       ...       ...   
931097  9           ...       0.0       0.0       0.0       0.0       0.0   
        10          ...       0.0       0.0       0.0       0.0       0.0   
        11          ...  23.49215  9.498727  2.253628  9.376545  9.376545   
        12          ...  23.49215  9.498727  2.253628  9.376545  9.376545   
        13          ...  23.49215  9.498727  2.253628  9.376545  9.376545   

                       122510    123210  125087  125196  125197  
recipe  ingredient                                               
1746116 0           24.419212  0.023016     0.0     0.0     0.0  
        1                 0.0       0.0     0.0     0.0     0.0  
        2                 0.0       0.0     0.0     0.0     0.0  
        3                 0.0       0.0     0.0     0.0     0.0  
        4                 0.0       0.0     0.0     0.0     0.0  
...                       ...       ...     ...     ...     ...  
931097  9                 0.0       0.0     0.0     0.0     0.0  
        10                0.0       0.0     0.0     0.0     0.0  
        11          24.419212  0.023016     0.0     0.0     0.0  
        12          24.419212  0.023016     0.0     0.0     0.0  
        13          24.419212  0.023016     0.0     0.0     0.0  

[2450 rows x 714 columns]

In [10]:
ingredient_food_ids = ingredients_df.join(food_ids)[['food_id']]
recipe_food_ids = ingredient_food_ids.groupby('recipe')['food_id'].aggregate(list).apply(pd.Series)
recipe_food_ids = recipe_food_ids.iloc[:, :15]
recipe_food_ids

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
recipe,,,,,,,,,,,,,,,
222,825.0,1025.0,666.0,667.0,670.0,633.0,195.0,669.0,670.0,586.0,901.0,105.0,NaN,NaN,NaN
1703,665,633,685,<NA>,901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8981,633.0,667.0,670.0,654.0,54.0,666.0,825.0,670.0,54.0,NaN,NaN,NaN,NaN,NaN,NaN
13596,805.0,806.0,631.0,44.0,456.0,654.0,872.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17929,274.0,12.0,57.0,670.0,825.0,633.0,12.0,57.0,341.0,456.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2185822,171.0,674.0,885.0,685.0,941.0,8.0,61.0,54.0,40.0,40.0,67.0,700.0,40.0,670.0,666.0
2189489,872.0,670.0,951.0,195.0,825.0,745.0,44.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2195357,342.0,941.0,54.0,124.0,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#TODO: decide on best recipe size.

In [11]:
recipe_food_ids = recipe_food_ids \
    .fillna(special_token_idxs['unknown']) \
    .astype('int64') \
    .to_numpy()

recipe_food_ids

array([[ 825, 1025,  666, ..., 1107, 1107, 1107],
       [ 665,  633,  685, ..., 1107, 1107, 1107],
       [ 633,  667,  670, ..., 1107, 1107, 1107],
       ...,
       [ 342,  941,   54, ..., 1107, 1107, 1107],
       [ 941,    8,  546, ..., 1107, 1107, 1107],
       [ 907,  885,   40, ..., 1107, 1107, 1107]])

In [12]:
np.save('../../data/local/final/partial/recipe_food_ids/0.npy', recipe_food_ids)

# Creating special tokens

- `[PAD]` - vector of zeros
- `[MASK]` - vector of ones?

In [21]:
food_compounds = food_compounds_df.to_numpy().astype('float')
food_compounds.shape

(1106, 713)

In [23]:
food_compounds = np.append(food_compounds, np.zeros([1, food_compounds.shape[1]]), axis=0)
food_compounds = np.append(food_compounds, np.random.rand(1, food_compounds.shape[1]), axis=0)
food_compounds = np.append(food_compounds, np.ones([1, food_compounds.shape[1]]), axis=0)
food_compounds.shape

(1109, 713)

In [24]:
np.save('../../data/local/final/full/food_compounds/0.npy', food_compounds)

In [25]:
food_compounds[-2:]

array([[0.65660908, 0.18656551, 0.01272906, ..., 0.60764267, 0.09497185,
        0.60938257],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])

In [16]:
np.save('../../data/local/final/food_compounds/0.npy', food_compounds)

In [17]:
food_compounds

array([[0.        , 0.        , 0.10933333, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.10933333, ..., 0.        , 0.        ,
        1.22318925],
       [0.        , 0.        , 0.10933333, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])

# PyTorch Dataset & Dataloader

Here each batch will have `batch_size` recipes. Each of these recipes will be fed into the model as:
- x: food_ids with 15% of their foods masked with a `[MASK]` token. 
- y: food_ids without mask

This will result in a matrix of size `batch_size` x `recipe_size` x `n_features`.

Problems:

- Create numerous mask elements of each recipe.
    - Will this cause overfit (data leakage -> feeding the model the answer)?
    - Overfitting on large recipes (more possible mask combinations)?
- Decision on where to get vectors.
    - It's much easier to work with a dataset of tokens rather - so possibly in the model itself?

Now we want to create a dataset of recipes with masked ingredients. How can we do this without creating duplicates?

In [31]:
#| export
class MaskedRecipeDataset(Dataset):

    def __init__(self, recipes):
        self.recipes = recipes
    
    def __len__(self):
        return len(self.recipes) # should be able to have more than one here
    
    def create_recipe_mask(self, recipe):
        recipe_size = (recipe != special_token_idxs['pad']).sum()
        mask = torch.tensor(False)
        while mask.sum() == torch.tensor(0):
            rand = torch.rand(recipe_size)
            mask = (rand < 0.15).to(bool)
        recipe[:recipe_size][mask] = special_token_idxs['mask']
        return recipe
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx): idx = idx.to_list()
        recipe = self.recipes[idx]
        recipe = torch.tensor(recipe, dtype=torch.int)
        return (
            self.create_recipe_mask(recipe.clone()).type(torch.int), 
            recipe.type(torch.int)
        )

In [32]:
dataset = MaskedRecipeDataset(recipe_food_ids)
dataset[0]

(tensor([1107, 1025, 1107,  667,  670,  633, 1107,  669,  670,  586, 1107,  105,
         1106, 1106, 1106], dtype=torch.int32),
 tensor([ 825, 1025,  666,  667,  670,  633,  195,  669,  670,  586,  901,  105,
         1106, 1106, 1106], dtype=torch.int32))

In [33]:
assert special_token_idxs['mask'] in dataset[0][0] and special_token_idxs['mask'] not in dataset[0][1]

In [34]:
dataloader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=10)

In [35]:
xb, yb = next(iter(dataloader))
xb.shape, yb.shape, xb[0], yb[0]

(torch.Size([10, 15]),
 torch.Size([10, 15]),
 tensor([1107,  669,  995,  131,  962,   13, 1106, 1106, 1106, 1106, 1106, 1106,
         1106, 1106, 1106], dtype=torch.int32),
 tensor([ 664,  669,  995,  131,  962,   13, 1106, 1106, 1106, 1106, 1106, 1106,
         1106, 1106, 1106], dtype=torch.int32))

In [36]:
from nbdev import nbdev_export; nbdev_export()